In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import ShallowLearn.ImageHelper as ih

In [ ]:
from ShallowLearn.IndiceFeatures import get_feature_order
from ShallowLearn.band_mapping import band_mapping
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from ShallowLearn.Training import create_row_data_frame

In [ ]:
features = [i[0] for i in get_feature_order()]

In [ ]:
features.append("mask")

In [ ]:
features = list(band_mapping.keys()) + features

In [ ]:
row_dataset, original_shape = create_row_data_frame("/media/ziad/Expansion/Cleaned_Data_Directory/", "imgs.npy")

In [ ]:
row_dataset

In [ ]:
indices = np.load("/media/ziad/Expansion/Cleaned_Data_Directory/indices.npy")
masks = np.load("/media/ziad/Expansion/Cleaned_Data_Directory/masks.npy")
images = np.load("/media/ziad/Expansion/Cleaned_Data_Directory/imgs.npy")

In [ ]:
masks = np.uint8(masks)

In [ ]:
data_combined = np.concatenate((images, indices, masks),axis = 3)

In [ ]:
data_combined.shape

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler

# Define the columns for each transformer
power_transformer_columns = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12','calculate_water_surface_index']
standard_scaler_columns = ['bgr', 'ci', 'ndci', 'oci', 'ssi', 'ti', 'wqi']
minmax_scaler_columns = ['B01', 'calculate_pseudo_subsurface_depth']
passthrough_columns = ['mask']
# Create transformers
power_transformer_transformer = ('power_transformer', PowerTransformer(), power_transformer_columns)
standard_scaler_transformer = ('standard_scaler', StandardScaler(), standard_scaler_columns)
minmax_scaler_transformer = ('minmax_scaler', MinMaxScaler(), minmax_scaler_columns)
passthrough_transformer = ('passthrough', 'passthrough', passthrough_columns)
# Initialize ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        power_transformer_transformer,
        standard_scaler_transformer,
        minmax_scaler_transformer,
        passthrough_transformer
    ]
)

# Initialize Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:

pipeline.get_feature_names_out()

In [ ]:
data_expanded = data_combined.reshape(-1, len(features))
df = pd.DataFrame(data_expanded, columns = features).dropna()


In [ ]:
reshaped_back = data_expanded.reshape(54 ,  data_combined.shape[1], data_combined.shape[2], len(features))

In [ ]:
ih.plot_rgb(reshaped_back[0], plot= True)

In [ ]:
transformed_data = pd.DataFrame(pipeline.fit_transform(df), columns = pipeline.feature_names_in_)

In [ ]:
transformed_data.duplicated().sum()

In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.pipeline import Pipeline
import umap

# Here we assume you have a data X
# X = your_data

# Pipeline for PCA followed by t-SNE
pipeline_pca_tsne = Pipeline([
    ('pca', PCA(n_components=5)),  # reduce dimensionality before t-SNE
    ('tsne', TSNE(n_components=2))  # you usually want 2D or 3D for visualization
])





In [ ]:
transformed_data = pipeline_pca_tsne.fit_transform(row_dataset)

In [ ]:
plt.plot(transformed_data[:,0], transformed_data[:,1], 'o')

In [ ]:
pca_model = PCA(n_components=30)
transformed_data = pca_model.fit_transform(row_dataset)

In [ ]:
# Extract explained variance ratio from pca_modified
explained_variance_ratio = pca_model.explained_variance_ratio_

# Create a cumulative sum of explained variance ratio
cumulative_explained_variance = explained_variance_ratio.cumsum()

# Plot the individual and cumulative explained variance ratios
plt.figure(figsize=(10, 5))

# Individual explained variance ratio
plt.bar(range(len(explained_variance_ratio)), explained_variance_ratio, alpha=0.5, label='Individual Explained Variance')

# Cumulative explained variance ratio
plt.step(range(len(cumulative_explained_variance)), cumulative_explained_variance, where='mid', label='Cumulative Explained Variance', color='red')

plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
original_images = row_dataset.to_numpy().reshape(original_shape)

In [ ]:
plt.plot(transformed_data[:,0], transformed_data[:,1], 'o')

In [ ]:
pca_model

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
#set figure size
#import kmeans
from sklearn.cluster import KMeans

kmeans_pca = KMeans(n_clusters=4)  # or however many clusters you want

kmeans_pca.fit(transformed_data)
labels_umap_pca = kmeans_pca.labels_


plt.figure(figsize=(15,25))

plt.scatter(transformed_data[:,0], transformed_data[:,1], c=labels_umap_pca, cmap='rainbow')
plt.title('PCs on whole Images after correction')
plt.ticklabel_format(style='plain', axis='x',useOffset=False)

# Plot image on top of the scatter plot in the location of the point 
for index, i in enumerate(labels_umap_pca):
    img = ih.plot_rgb(original_images[index])
    im = OffsetImage(img, zoom=0.2)  # Adjust zoom here
    ab = AnnotationBbox(im, (transformed_data[index,0], transformed_data[index,1]), box_alignment=(0.5, 0.5), 
                        bboxprops = dict(edgecolor='none', alpha=0.5, boxstyle="square,pad=0"))
    plt.gca().add_artist(ab)

# Print labels for each point
label_dict = {}
for i, txt in enumerate(labels_umap_pca):
    # Only print one label for each cluster
    if txt not in label_dict:
        label_dict[txt] = 1
        plt.annotate(txt, (transformed_data[i,0], transformed_data[i,1]), color = 'red')


plt.show()


In [ ]:
#import ImageHelper as ih

def plot_pca_with_images(transformed_data, original_images, n_clusters=4, figsize=(15,25), zoom_level=0.2):
    # Perform KMeans clustering
    kmeans_pca = KMeans(n_clusters=n_clusters)
    kmeans_pca.fit(transformed_data)
    labels_umap_pca = kmeans_pca.labels_
    
    # Set figure size and plot scatter points
    plt.figure(figsize=figsize)
    plt.scatter(transformed_data[:,0], transformed_data[:,1], c=labels_umap_pca, cmap='rainbow')
    plt.title('PCs on whole Images after correction')
    plt.ticklabel_format(style='plain', axis='x', useOffset=False)
    
    # Overlay images on scatter plot
    for index, i in enumerate(labels_umap_pca):
        img = ih.plot_rgb(original_images[index])
        im = OffsetImage(img, zoom=zoom_level)
        ab = AnnotationBbox(im, (transformed_data[index,0], transformed_data[index,1]), box_alignment=(0.5, 0.5), 
                            bboxprops=dict(edgecolor='none', alpha=0.5, boxstyle="square,pad=0"))
        plt.gca().add_artist(ab)
    
    # Print labels for each point
    label_dict = {}
    for i, txt in enumerate(labels_umap_pca):
        if txt not in label_dict:
            label_dict[txt] = 1
            plt.annotate(txt, (transformed_data[i,0], transformed_data[i,1]), color='red')
    
    plt.show()


In [ ]:
plot_pca_with_images(transformed_data, original_images)

In [ ]:
plt.figure(figsize=(15,25))

plt.scatter(transformed_data[:,0], transformed_data[:,1], c=labels_umap_pca, cmap='rainbow')
plt.title('PCs on whole Images after correction')
plt.ticklabel_format(style='plain', axis='x',useOffset=False)

In [ ]:
from ShallowLearn.RadiometricNormalisation import histogram_matching, pca_based_normalization, pca_filter_and_normalize

In [ ]:
ref = original_images[1]
plt.title(kmeans_pca.predict(transformed_data[0].reshape(2, -1).T))
ih.plot_rgb(ref, plot = True)


In [ ]:
transformed_data[0].reshape(-1)

In [ ]:
modified_dataset = []

pcs_after_tf = []
for index, label in enumerate(labels_umap_pca):
    if label == 0 or label == 2:
        normalised_img = pca_based_normalization(original_images[index], ref)
        transformed_img = pca_model.transform(normalised_img.reshape(1, -1))
        pcs_after_tf.append(transformed_img)
        fig, ax = plt.subplots(1, 2, figsize = (10, 10))
        plt.title(kmeans_pca.predict(transformed_data[index].reshape(2, -1).T))
        ax[0].set_title("Original image")
        ax[1].set_title("Normalised image")
        ax[0].imshow(ih.plot_rgb(original_images[index]))
        ax[1].imshow(ih.plot_rgb(normalised_img))
        modified_dataset.append(normalised_img)
    else:
        modified_dataset.append(original_images[index])
        plt.show()

In [ ]:
modified_array = np.array(modified_dataset)

In [ ]:
np.save("/media/ziad/Expansion/Cleaned_Data_Directory/radiometrically_normalized.npy", modified_array)

In [ ]:
pcs_after_tf

In [ ]:
pcs_after_tf = np.array(pcs_after_tf).reshape(-1, 2)

In [ ]:
pcs_after_tf

In [ ]:


# # Pipeline for PCA followed by t-SNE
# pipeline_pca_tsne = Pipeline([
#     ('pca', PCA(n_components=5)),  # reduce dimensionality before t-SNE
#     ('tsne', TSNE(n_components=2))  # you usually want 2D or 3D for visualization
# ])


pca_modified= PCA(n_components=30)

pca_modified_array = pca_modified.fit_transform(modified_array.reshape(54, -1))
print(pca_modified.explained_variance_ratio_)
plt.figure(figsize=(15,25))

plt.scatter(pca_modified_array[:,0], pca_modified_array[:,1], c=labels_umap_pca, cmap='rainbow')
#set log scale
plt.gca().set_yscale('symlog')
plt.gca().set_xscale('symlog')

plt.title('PCs on whole Images after correction')
# plt.ticklabel_format(style='plain', axis='x',useOffset=False)

In [ ]:
# Extract explained variance ratio from pca_modified
explained_variance_ratio = pca_modified.explained_variance_ratio_

# Create a cumulative sum of explained variance ratio
cumulative_explained_variance = explained_variance_ratio.cumsum()

# Plot the individual and cumulative explained variance ratios
plt.figure(figsize=(10, 5))

# Individual explained variance ratio
plt.bar(range(len(explained_variance_ratio)), explained_variance_ratio, alpha=0.5, label='Individual Explained Variance')

# Cumulative explained variance ratio
plt.step(range(len(cumulative_explained_variance)), cumulative_explained_variance, where='mid', label='Cumulative Explained Variance', color='red')

plt.ylabel('Explained Variance Ratio')
plt.xlabel('Principal Components')
plt.legend(loc='best')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15,25))

plt.scatter(transformed_data[:,0], transformed_data[:,1], c=labels_umap_pca, cmap='rainbow')
plt.scatter(pcs_after_tf[:,0], pcs_after_tf[:,1], c='black', cmap='rainbow')
#set log scale
plt.gca().set_yscale('symlog')
plt.gca().set_xscale('symlog')

plt.title('PCs on whole Images after correction')
# plt.ticklabel_format(style='plain', axis='x',useOffset=False)

In [ ]:
# Pipeline for PCA followed by UMAP
pipeline_pca_umap = Pipeline([
    ('pca', PCA(n_components=5)),  # reduce dimensionality before UMAP
    ('umap', umap.UMAP(n_components=2))  # you usually want 2D or 3D for visualization
])

# Pipeline for just PCA
pipeline_pca = Pipeline([
    ('pca', PCA(n_components=2))  # or however many components you want
])


In [ ]:
image_rows = images.reshape(54, -1)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()
image_rows = scaler.fit_transform(image_rows)

In [ ]:
pca = PCA(n_components=10)
pca_X = pca.fit_transform(image_rows)

explained_variance = pca.explained_variance_ratio_

print(explained_variance)

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

kmeans_pca_X = KMeans(n_clusters=3)  # or however many clusters you want

kmeans_pca_X.fit(pca_X)
labels_pca = kmeans_pca_X.labels_



In [ ]:
plt.figure(figsize=(10,10))
plt.title("Scaled PCs with whole images as input")
plt.scatter(pca_X[:,0], pca_X[:,1],c=labels_pca,cmap='viridis')
plt.gca().set_xlabel("PC0")
plt.gca().set_ylabel("PC1")

In [ ]:
umap_model =  umap.UMAP(n_components=2)

In [ ]:
embedding = umap_model.fit_transform(pca_X)

In [ ]:

plt.plot(embedding[:,0], embedding[:,1], 'o', markersize=2, color='blue', alpha=0.5, label='embedding')


In [ ]:

kmeans_image_rows = KMeans(n_clusters=2)  # or however many clusters you want

kmeans_image_rows.fit(embedding)
labels_pca_umap = kmeans_image_rows.labels_

# Create a 2D scatter plot for PCA -> t-SNE with points colored by their cluster membership
plt.figure(figsize=(10, 7))
plt.scatter(embedding[:, 0], embedding[:, 1], c=labels_pca_umap, s=10, cmap='viridis')
plt.title('PCA followed by UMAP with KMeans Clustering')
plt.show()

In [ ]:
from ShallowLearn import Transform as tf

In [ ]:
fixed_imagery = []
first_one = True
only_rgb = []
for index, i in enumerate(labels_pca):
    # plt.imshow(ih.plot_rgb(images[index]))
    # plt.title(i)
    # plt.show()

    if i == 2 or i == 1:
        if first_one:
            first_one = False
            continue
        fig, ax = plt.subplots(1, 3, figsize=(10, 5))
        ax[0].imshow(ih.plot_rgb(images[index]))
        ax[0].set_title(f"Original Cropped Image")
        lab_image = ih.plot_rgb(tf.transform_multiband_lab(images[index]))
        ax[1].imshow(lab_image)
        mask = np.any(lab_image != 0, axis = 2)
        ax[1].set_title(f"Restretched Image (LAB)")
        
        restreched_hsv = tf.transform_multiband_hsv(tf.transform_multiband_lab(images[index]), max_value = 3.2)
        restreched_hsv = ih.generate_multichannel_mask(restreched_hsv, mask)
        restreched_hsv = ih.generate_multichannel_mask(restreched_hsv, mask)
        # restreched_hsv = np.where(restreched_hsv<0.001, np.nan, restreched_hsv)
        ax[2].imshow(ih.plot_rgb(restreched_hsv))
        ax[2].set_title(f"Restretched Image (LAB -> HSV)")
        
        fixed_imagery.append(restreched_hsv)
        plt.suptitle(f"Restretched Image")
        plt.show()
        only_rgb.append(ih.plot_rgb(restreched_hsv))
    elif i == 0:
        fixed_imagery.append(images[index])
        only_rgb.append(ih.plot_rgb(images[index]))


In [ ]:
fixed_imagery = np.array(fixed_imagery)

In [ ]:
ih.plot_histograms(fixed_imagery[-1])

In [ ]:
only_rgb = np.array(only_rgb)


In [ ]:
only_rgb = only_rgb.reshape(len(only_rgb), -1)

In [ ]:
image_rows = fixed_imagery.reshape(len(fixed_imagery), -1)

In [ ]:
cleaned_rows = pd.DataFrame(image_rows).dropna(axis = 1)

In [ ]:
cleaned_rows.replace(0, np.nan, inplace=True)

In [ ]:
cleaned_rows.dropna(axis = 1, inplace = True)

In [ ]:
pca = PCA(n_components=10)
pca_X = pca.fit_transform(only_rgb)

explained_variance = pca.explained_variance_ratio_

print(explained_variance)

In [ ]:
pca = PCA(n_components=10)
pca_X = pca.fit_transform(cleaned_rows)

explained_variance = pca.explained_variance_ratio_

print(explained_variance)


In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

kmeans_pca_X = KMeans(n_clusters=5)  # or however many clusters you want

kmeans_pca_X.fit(pca_X)
labels_pca = kmeans_pca_X.labels_



In [ ]:
umap_model = umap.UMAP(n_neighbors=5, min_dist=0.3, metric='correlation')
umap_model.fit_transform(pca_X)
plt.scatter(umap_model.embedding_[:, 0], umap_model.embedding_[:, 1])

In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
#set figure size
kmeans_umap_pca_X = KMeans(n_clusters=4)  # or however many clusters you want

kmeans_umap_pca_X.fit(pca_X)
labels_umap_pca = kmeans_umap_pca_X.labels_


plt.figure(figsize=(15,25))

plt.scatter(umap_model.embedding_[:,0], umap_model.embedding_[:,1], c=labels_pca, cmap='rainbow')
plt.title('PCs on whole Images after correction')
plt.ticklabel_format(style='plain', axis='x',useOffset=False)

# Plot image on top of the scatter plot in the location of the point 
for index, i in enumerate(labels_umap_pca):
    img = ih.plot_rgb(fixed_imagery[index])
    im = OffsetImage(img, zoom=0.2)  # Adjust zoom here
    ab = AnnotationBbox(im, (umap_model.embedding_[index,0], umap_model.embedding_[index,1]), box_alignment=(0.5, 0.5), 
                        bboxprops = dict(edgecolor='none', alpha=0.5, boxstyle="square,pad=0"))
    plt.gca().add_artist(ab)

# Print labels for each point
label_dict = {}
for i, txt in enumerate(labels_umap_pca):
    # Only print one label for each cluster
    if txt not in label_dict:
        label_dict[txt] = 1
        plt.annotate(txt, (umap_model.embedding_[i,0], umap_model.embedding_[i,1]), color = 'red')


plt.show()


In [ ]:
from matplotlib.offsetbox import OffsetImage, AnnotationBbox
#set figure size
plt.figure(figsize=(15,25))

plt.scatter(pca_X[:,0], pca_X[:,1], c=labels_pca, cmap='rainbow')
plt.title('PCs on whole Images after correction')
plt.ticklabel_format(style='plain', axis='x',useOffset=False)

# Plot image on top of the scatter plot in the location of the point 
for index, i in enumerate(labels_pca):
    img = ih.plot_rgb(fixed_imagery[index])
    im = OffsetImage(img, zoom=0.2)  # Adjust zoom here
    ab = AnnotationBbox(im, (pca_X[index,0], pca_X[index,1]), box_alignment=(0.5, 0.5), 
                        bboxprops = dict(edgecolor='none', alpha=0.0, boxstyle="square,pad=0"))
    plt.gca().add_artist(ab)

# Print labels for each point
label_dict = {}
for i, txt in enumerate(labels_pca):
    # Only print one label for each cluster
    if txt not in label_dict:
        label_dict[txt] = 1
        plt.annotate(txt, (pca_X[i,0], pca_X[i,1]), color = 'red')


plt.show()


In [ ]:
for index, i in enumerate(labels_pca):
    plt.imshow(ih.plot_rgb(fixed_imagery[index]))
    plt.title(i)
    plt.show()

In [ ]:
from ShallowLearn.IndiceFeatures import GenerateIndicesPerImage as gpi

In [ ]:
images_w_indices = []
for image in fixed_imagery:
    images_w_indices.append(gpi(image).indices)

In [ ]:
fixed_imagery = np.concatenate((fixed_imagery, np.array(images_w_indices)), axis = 3)

In [ ]:
fixed_imagery

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PowerTransformer, MinMaxScaler

# Define the columns for each transformer
power_transformer_columns = ['B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B10', 'B11', 'B12','calculate_water_surface_index']
standard_scaler_columns = ['bgr', 'ci', 'ndci', 'oci', 'ssi', 'ti', 'wqi']
minmax_scaler_columns = ['B01', 'calculate_pseudo_subsurface_depth']
# Create transformers
power_transformer_transformer = ('power_transformer', PowerTransformer(), power_transformer_columns)
standard_scaler_transformer = ('standard_scaler', StandardScaler(), standard_scaler_columns)
minmax_scaler_transformer = ('minmax_scaler', MinMaxScaler(), minmax_scaler_columns)
# Initialize ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        power_transformer_transformer,
        standard_scaler_transformer,
        minmax_scaler_transformer
        ]
)

# Initialize Pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
del features[-1]

In [ ]:
fixed_imagery_copy = fixed_imagery.reshape(-1, len(features))
df = pd.DataFrame(fixed_imagery_copy, columns = features).dropna()
transformed_data = pipeline.fit_transform(df)

In [ ]:
transformed_data = pd.DataFrame(transformed_data, columns = pipeline.feature_names_in_)

In [ ]:
#X_pca_tsne = pipeline_pca_tsne.fit_transform(transformed_data)


In [ ]:
#np.save('/media/ziad/Expansion/Cleaned_Data_Directory/X_5pca_2tsne.npy', X_pca_tsne)

In [ ]:
transformed_data.to_csv("../Data/transformed_data.csv")

In [ ]:
pca = PCA(n_components=2)
pca_data = pca.fit_transform(transformed_data)

plt.scatter(pca_data[:,0], pca_data[:,1])

In [ ]:
import seaborn as sns

In [ ]:
pca.get_covariance()
#make a covariance plot of the first two principal components
plt.figure(figsize=(20,20))
covariance = pd.DataFrame(pca.get_covariance(), columns = features).sort_values(by = "B02", ascending = False)
sns.heatmap(covariance, annot=True, fmt='.2f', cmap='Blues',xticklabels=features, yticklabels=features)
plt.show()

In [ ]:
reduced_data = transformed_data[['B04','B03','B02','B08']]

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca_data = pca.fit_transform(transformed_data)




In [ ]:
from ShallowLearn.EmbeddingVis import convert_colour_space

In [ ]:
color_col = convert_colour_space(reduced_data, ["B04", "B03", "B02"], alpha = 0.1)

In [ ]:
color_col

In [ ]:
reduced_data.describe()

In [ ]:
import math

def reshape_to_square_v2(image):
    # Calculate the total number of elements divided by the channel size (4 in this case)
    total_elements = image.size // image.shape[-1]
    
    # Find n
    n = math.ceil(math.sqrt(total_elements))
    
    # Create a zero-filled reshaped_image
    reshaped_image = np.zeros((n, n, 4))
    
    # Flatten the original image for easier indexing
    flat_image = image.reshape(-1, image.shape[-1])
    
    # Fill the reshaped_image
    for idx, row in enumerate(flat_image):
        i = idx // n
        j = idx % n
        reshaped_image[i, j] = row
        
    return reshaped_image


In [ ]:
# scaler = MinMaxScaler(feature_range=(0,255))
# reduced_copy = scaler.fit_transform(reduced_data.copy())

In [ ]:
img = reshape_to_square_v2(np.expand_dims(np.array(reduced_data), axis = 1))

In [ ]:
def sort_multidimensional_data(data):
    """
    Sorts a multidimensional array based on the average value of the last dimension.

    Parameters:
    - data (ndarray): A multidimensional numpy array.

    Returns:
    - ndarray: The data sorted by the average value of the last dimension.
    """

    # Compute the 'sorting criterion' as the mean along the last axis
    sorting_criterion = np.mean(data, axis=-1)

    # Get sorting indices from sorting criterion
    sorting_indices = np.argsort(sorting_criterion.ravel())

    # Flatten and sort the original data with these indices
    sorted_flattened = data.reshape(-1, data.shape[-1])[sorting_indices]

    # Reshape to get the sorted data
    sorted_data = sorted_flattened.reshape(data.shape)
    
    return sorted_data

In [ ]:
sorted_image = sort_multidimensional_data(img)

In [ ]:
plt.imshow(tf.transform_lab_stretch(sorted_image)[:,:,:3])

In [ ]:
scaled_image = np.uint8(tf.LCE_multi(sorted_image))

In [ ]:
plt.imshow(sorted_image[:,:,:4])

In [ ]:
plt.imshow(sorted_image[:,:,:4])

In [ ]:
color_col

In [ ]:
# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(pca_data[:,0], pca_data[:,1], c=color_col)  # Scale size as needed
plt.xlabel('PC0')
plt.ylabel('PC1')
# plt.xlim(-3,4)
# plt.ylim(-3,2)
plt.title('2D PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(pca_data[:,0], pca_data[:,1], c=color_col)  # Scale size as needed
plt.xlabel('PC0')
plt.ylabel('PC1')
plt.xlim(-20,20)
plt.ylim(-20,20)
plt.title('2D PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
reduced_copy = reduced_data.copy()
# Ensure all values are within the range of -1 to 1
for column in ['B04', 'B03', 'B02', 'B08']:
    reduced_copy[column] = np.clip(reduced_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    reduced_copy[column] = (reduced_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
reduced_copy['color'] = list(zip(reduced_copy.B04, reduced_copy.B03, reduced_copy.B02, reduced_copy.B08 / 10))



In [ ]:
# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(pca_data[:,0], pca_data[:,1], c=reduced_copy['color'], s=reduced_copy['B08']*100)  # Scale size as needed
plt.xlabel('PC0')
plt.ylabel('PC1')
plt.xlim(-3,4)
plt.ylim(-3,2)
plt.title('2D PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:

umap_model =  umap.UMAP(min_dist = 1, n_neighbors = 300 , random_state = 42)
sample = transformed_data.sample(200_000)
embedding = umap_model.fit_transform(sample)

# Plot the results


In [ ]:

sample_copy = sample.copy()

In [ ]:

sample = sample_copy.copy()
# Ensure all values are within the range of -1 to 1
for column in ['B04', 'B03', 'B02', 'B08']:
    sample[column] = np.clip(sample[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample[column] = (sample[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample['color'] = list(zip(sample.B04, sample.B03, sample.B02, sample.B08 / 10))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(embedding[:,0], embedding[:,1], c=sample['color'], s=sample['B08']*100)  # Scale size as needed
plt.title('2D Umap embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
pca = PCA(n_components=5)
pca_data = pca.fit(transformed_data)

In [ ]:
sample = transformed_data.sample(200_000, random_state = 42)

umap_model =  umap.UMAP(min_dist = 1, n_neighbors = 300 , random_state = 42)
embedding = umap_model.fit_transform(pca.transform(sample))

# Plot the results

#sample = sample_copy.copy()

In [ ]:
sample_copy = sample.copy()

for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = np.clip(sample_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = (sample_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample_copy['color'] = list(zip(sample_copy.B04, sample_copy.B03, sample_copy.B02, sample_copy.B08 / 10))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(embedding[:,0], embedding[:,1], c=sample_copy['color'], s=sample_copy['B08']*100)  # Scale size as needed
plt.title('2D Umap embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
sample = transformed_data.sample(200_000, random_state = 42)


In [ ]:

umap_model =  umap.UMAP(min_dist = 1, n_neighbors = 300 , random_state = 42, n_components = 10)
embedding = umap_model.fit_transform(sample)







In [ ]:

pca = PCA(n_components=10)
pca_data = pca.fit_transform(embedding)

In [ ]:
print(pca.explained_variance_ratio_)


In [ ]:
col = convert_colour_space(sample)

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(pca_data[:,0], pca_data[:,1], c=col)  # Scale size as needed
plt.title('2D UMAP to PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
sample_copy = sample.copy()

for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = np.clip(sample_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = (sample_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample_copy['color'] = list(zip(sample_copy.B04, sample_copy.B03, sample_copy.B02, sample_copy.B08 / 10))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(pca_data[:,0], pca_data[:,1], c=sample_copy['color'], s=sample_copy['B08']*100)  # Scale size as needed
plt.title('2D UMAP to PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
pca_data.shape

In [ ]:
sample = transformed_data.sample(500_000, random_state = 42)

pca = PCA(n_components=10)
pca_data = pca.fit_transform(sample)

In [ ]:
from sklearn import manifold
# tsne = manifold.TSNE(n_components = 2,learning_rate=400, perplexity = 25, random_state = 42, verbose = 2, n_iter = 2000, n_jobs=-1)
# transformed_data = tsne.fit_transform(pca_data)
tsne = manifold.TSNE(n_components = 2,learning_rate=500, perplexity = 25, random_state = 42, verbose = 2, n_iter = 2000, n_jobs=-1)
tsne_data = tsne.fit_transform(pca_data)

In [ ]:
sample_copy = sample.copy()

for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = np.clip(sample_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = (sample_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample_copy['color'] = list(zip(sample_copy.B04, sample_copy.B03, sample_copy.B02, sample_copy.B08 / 10))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(tsne_data[:,0], tsne_data[:,1], c=sample_copy['color'], s=sample_copy['B08']*10)  # Scale size as needed
plt.title('2D PCA embeddings to t-sne with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
sample_copy = sample.copy()

for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = np.clip(sample_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = (sample_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample_copy['color'] = list(zip(sample_copy.B04, sample_copy.B03, sample_copy.B02))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(tsne_data[:,0], tsne_data[:,1], c=sample_copy['color'])  # Scale size as needed
plt.title('2D UMAP to PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:
sample_copy = sample.copy()

for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = np.clip(sample_copy[column], -1, 1)

# Convert the values in the columns from the range of -1 to 1 to 0 to 1 (since RGB values and alpha are in the range 0 to 1)
for column in ['B04', 'B03', 'B02', 'B08']:
    sample_copy[column] = (sample_copy[column] + 1) / 2

# Create a new column 'color' in dataframe that will contain RGBA colors for each row
sample_copy['color'] = list(zip(sample_copy.B04, sample_copy.B03, sample_copy.B02, sample_copy.B08 / 10))

# Create scatter plot with size proportional to 'B08' value
plt.figure(figsize=(10, 10))
plt.scatter(tsne_data[:,0], tsne_data[:,1], c=sample_copy['color'], s=sample_copy['B08']*100)  # Scale size as needed
plt.title('2D UMAP to PCA embeddings with RGB colors (B4,B3,B2) and size proportional to B08')
plt.show()

In [ ]:

# Fit a KMeans model to the PCA -> t-SNE data
kmeans_pca = KMeans(n_clusters=10)  # or however many clusters you want
kmeans_pca.fit(tsne_data)
labels_pca_tsne = kmeans_pca.labels_

# Create a 2D scatter plot for PCA -> t-SNE with points colored by their cluster membership
plt.figure(figsize=(10, 10))

# Find unique labels and their corresponding colors
unique_labels = list(set(labels_pca_tsne))
colors = plt.cm.viridis(np.linspace(0, 1, len(unique_labels)))

for i, label in enumerate(unique_labels):
    plt.scatter(tsne_data[labels_pca_tsne == label, 0], 
                tsne_data[labels_pca_tsne == label, 1], 
                color=colors[i], 
                label=f'Cluster {label}', 
                s=10)

plt.legend()
plt.title('PCA to t-sne followed by KMeans Clustering')
plt.show()

In [ ]:

X_pca_umap = pipeline_pca_umap.fit_transform(reduced_data.sample(300_000, random_state=42))



In [ ]:


# Fit a KMeans model to the PCA -> t-SNE data
kmeans_pca_tsne = KMeans(n_clusters=10)  # or however many clusters you want
kmeans_pca_tsne.fit(X_pca_umap)
labels_pca_tsne = kmeans_pca_tsne.labels_

# Create a 2D scatter plot for PCA -> t-SNE with points colored by their cluster membership
plt.figure(figsize=(10, 7))
plt.scatter(X_pca_umap[:, 0], X_pca_umap[:, 1], c=labels_pca_tsne, s=10, cmap='viridis')
plt.title('PCA followed by umap with KMeans Clustering')
plt.show()

In [ ]:


# Fit a KMeans model to the PCA -> t-SNE data
kmeans_pca_tsne = KMeans(n_clusters=8)  # or however many clusters you want
kmeans_pca_tsne.fit(X_pca_umap)
labels_pca_tsne = kmeans_pca_tsne.labels_

# Create a 2D scatter plot for PCA -> t-SNE with points colored by their cluster membership
plt.figure(figsize=(10, 7))
plt.scatter(X_pca_umap[:, 0], X_pca_umap[:, 1], c=labels_pca_tsne, s=10, cmap='viridis')
plt.title('PCA followed by t-SNE with KMeans Clustering')
plt.show()

In [ ]:
X_pca = pipeline_pca.fit_transform(transformed_data)

In [ ]:
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans

# Fit a KMeans model to the PCA -> t-SNE data
kmeans_pca_tsne = KMeans(n_clusters=3)  # or however many clusters you want
kmeans_pca_tsne.fit(X_pca_tsne)
labels_pca_tsne = kmeans_pca_tsne.labels_

# Create a 2D scatter plot for PCA -> t-SNE with points colored by their cluster membership
plt.figure(figsize=(10, 7))
plt.scatter(X_pca_tsne[:, 0], X_pca_tsne[:, 1], c=labels_pca_tsne, s=10, cmap='viridis')
plt.title('PCA followed by t-SNE with KMeans Clustering')
plt.show()

In [ ]:
from xgboost import XGBClassifier
#import test train split
from sklearn.model_selection import train_test_split
#import metrics accuracy 
from sklearn.metrics import accuracy_score
# Assuming your preprocessed data is X and the kmeans labels are y
X = sample.copy()
y = labels_pca_tsne
#test train split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Create a new pipeline with XGBoost
# pipeline_xgb = Pipeline(steps=[('preprocessor', pipeline),
#                                ('classifier', XGBClassifier())])
clf = XGBClassifier()
# Fit the entire pipeline
clf.fit(X_train, y_train)
# Predict the labels of the test set
y_pred = clf.predict(X_test)
# Compute the accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
#import recall and precision
from sklearn.metrics import recall_score, precision_score
# test recall and precision
print('Recall:', recall_score(y_test, y_pred, average='weighted'))
print('Precision:', precision_score(y_test, y_pred, average='weighted'))

In [ ]:
data_combined[0].reshape(-1, 23).shape

In [ ]:
def predict_method(image):
    temp = image.copy()
    original_shape = image.shape
    temp = temp.reshape(-1, 23)
    temp = pd.DataFrame(temp, columns=pipeline.feature_names_in_)
    temp = pipeline.transform(temp)
    temp = pd.DataFrame(temp, columns=pipeline.feature_names_in_)
    #temp = temp[reduced_data.columns]
    pred = clf.predict(temp)
    pred = pred.reshape(original_shape[:-1] + (1,))
    return pred

In [ ]:
def transform_data(image):
    temp = image.copy()
    original_shape = temp.shape
    temp = temp.reshape(-1, temp.shape[-1])
    temp = pd.DataFrame(temp, columns=pipeline.feature_names_in_)
    temp = pipeline.transform(temp)
    temp = pd.DataFrame(temp, columns=pipeline.feature_names_in_)
    #temp = temp[reduced_data.columns]
    return temp 

In [ ]:
#bar plot of feature importance of the classifier 
pd.DataFrame(clf.feature_importances_, columns = ['Importance'], index = pipeline.feature_names_in_).sort_values(by = 'Importance', ascending = False).plot(kind = 'bar')

In [ ]:
fixed_imagery

In [ ]:
pd.DataFrame(labels_pca)

In [ ]:
fixed_imagery[img_no].shape

In [ ]:
img_no = 14
ih.plot_rgb(data_combined[img_no], plot = True)

In [ ]:
ih.discrete_implot(predict_method(data_combined[img_no]))

In [ ]:
# Create a 2D scatter plot for PCA -> UMAP
plt.figure(figsize=(10, 7))
plt.scatter(X_pca_umap[:, 0], X_pca_umap[:, 1], s=10)
plt.title('PCA followed by UMAP')
plt.show()

# Create a 2D scatter plot for PCA
plt.figure(figsize=(10, 7))
plt.scatter(X_pca[:, 0], X_pca[:, 1], s=10)
plt.title('PCA')
plt.show()

In [ ]:
import shap

In [ ]:
shap.initjs()

In [ ]:

explainer = shap.TreeExplainer(clf)



In [ ]:
transformed_image_single = transform_data(data_combined[img_no])

In [ ]:
# Calculate SHAP values

shap_values = explainer.shap_values(transformed_image_single)


In [ ]:

# Plot summary of SHAP values using matplotlib
shap.summary_plot(shap_values, transformed_image_single)

In [ ]:
len(shap_values)

In [ ]:
original_shape = data_combined[img_no].shape

In [ ]:
original_shape

In [ ]:
shap_values[0].shape

In [ ]:
pd.DataFrame(features)

In [ ]:
reshaped_shap = select_channels(shap_values[0].reshape(original_shape[0], original_shape[1], 23), [14,12,19])

In [ ]:
shap_vals = shap_values[0].reshape(original_shape[0], original_shape[1], 23)
for i in range(23):
    plt.title(features[i]   + " SHAP Values")
    plt.imshow(shap_vals[:, :, i])
    plt.colorbar()
    plt.show()

In [ ]:
plt.imshow(shap_values[1].reshape(original_shape[0], original_shape[1], 4)[:,:,:])

In [ ]:
scaler = MinMaxScaler()
for i in range(10):
    plt.imshow(scaler.fit_transform(shap_values[i]).reshape(original_shape[0], original_shape[1], 4)[:,:,:3])
    plt.colorbar()
    plt.show()

In [ ]:
plt.imshow(scaler.fit_transform(shap_values[i]).reshape(original_shape[0], original_shape[1], 4)[:,:,3])
plt.colorbar()
plt.show()

In [ ]:
# Calculate SHAP values
transformed_image_single = sorted_image.reshape(-1, 4)
shap_values = explainer.shap_values(transformed_image_single)


In [ ]:
scaler = MinMaxScaler()
original_shape = sorted_image.shape
for i in range(10):
    plt.imshow(scaler.fit_transform(shap_values[i]).reshape(original_shape[0], original_shape[1], 4)[:,:,:4])
    plt.show()

In [ ]:
scaler = MinMaxScaler()
original_shape = sorted_image.shape
for i in range(10):
    plt.imshow(scaler.fit_transform(shap_values[i]).reshape(original_shape[0], original_shape[1], 4)[:,:,:4])
    plt.show()